# Market Manipulation Simulations

This notebook analyzes the economic feasibility and defenses against market manipulation attempts. We simulate various manipulation strategies to understand vulnerabilities and design effective countermeasures.

## Executive Summary

Market manipulation poses significant risks to perpetual markets:
- **Spot Price Manipulation**: Moving underlying prices to trigger liquidations
- **Funding Rate Gaming**: Creating imbalances to extract funding payments
- **Oracle Attacks**: Exploiting price feed vulnerabilities
- **Liquidity Squeezes**: Removing depth to amplify price movements

Understanding these attack vectors helps design robust defenses.

In [ ]:
# Essential imports
import sys
sys.path.append('../src')

from risk_model.notebook_helpers import fetch_market_data
from risk_model.chart_config import setup_chart_style, COLORS
from risk_model.scenarios import (
    calculate_manipulation_cost, analyze_manipulation_pnl,
    analyze_funding_manipulation, analyze_oracle_defense
)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

setup_chart_style()

## 1. Manipulation Economics Framework

### Cost-Benefit Analysis

For manipulation to be profitable:
```
Profit from Position > Cost of Manipulation + Funding Costs + Risk Premium
```

Key factors affecting profitability:
1. **Market Depth**: Deeper markets cost more to manipulate
2. **Position Leverage**: Higher leverage amplifies profits
3. **Time Horizon**: Longer manipulation periods increase funding costs
4. **Detection Risk**: Sophisticated monitoring increases required risk premium

### Attack Vectors

| Vector | Method | Cost | Impact | Defense |
|--------|--------|------|--------|----------|
| Spot Push | Buy/sell spot aggressively | High | Direct | Deep liquidity |
| Cascade Trigger | Target liquidation levels | Medium | Amplified | Position limits |
| Oracle Gaming | Manipulate price feeds | Variable | Systemic | Multi-venue oracles |
| Funding Squeeze | Create position imbalance | Low | Sustained | Rate caps |

In [ ]:
# Load market data
from risk_model.config import load_markets_config
markets = load_markets_config()
market_data = fetch_market_data(markets, lookback_days=7, limit_markets=5)

# Select test market
test_market = 'ETH-PERP'
if test_market not in market_data:
    test_market = list(market_data.keys())[0]
    
print(f"\n🎯 Analyzing manipulation scenarios for: {test_market}")

## 2. Spot Price Manipulation Analysis

We calculate the cost to move spot prices by various percentages and analyze potential profits from leveraged perpetual positions.

### Manipulation Cost Curve

The cost increases non-linearly with desired price impact due to:
- Consuming deeper orderbook levels
- Triggering arbitrageur response
- Activating market maker inventory

In [ ]:
# Calculate manipulation costs for different impacts
impacts = [0.5, 1, 2, 5, 10, 20]  # Percentage impacts
manipulation_costs = []

orderbook = market_data[test_market]['orderbook']

for impact in impacts:
    # Calculate for both directions
    up_cost = calculate_manipulation_cost(orderbook, impact, 'up')
    down_cost = calculate_manipulation_cost(orderbook, impact, 'down')
    
    manipulation_costs.append({
        'impact_pct': impact,
        'up_cost': up_cost['manipulation_cost_usd'],
        'down_cost': down_cost['manipulation_cost_usd'],
        'up_orders': up_cost['orders_consumed'],
        'down_orders': down_cost['orders_consumed']
    })

cost_df = pd.DataFrame(manipulation_costs)

# Visualize manipulation cost curve
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Cost curves
ax1.plot(cost_df['impact_pct'], cost_df['up_cost']/1000, 
         color=COLORS['success'], marker='o', linewidth=2.5, label='Push Up')
ax1.plot(cost_df['impact_pct'], cost_df['down_cost']/1000, 
         color=COLORS['danger'], marker='s', linewidth=2.5, label='Push Down')
ax1.set_xlabel('Target Price Impact (%)')
ax1.set_ylabel('Manipulation Cost ($000s)')
ax1.set_title('Cost to Manipulate Spot Price')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Orders consumed
width = 0.35
x = np.arange(len(impacts))
ax2.bar(x - width/2, cost_df['up_orders'], width, color=COLORS['success'], label='Push Up')
ax2.bar(x + width/2, cost_df['down_orders'], width, color=COLORS['danger'], label='Push Down')
ax2.set_xlabel('Target Price Impact (%)')
ax2.set_ylabel('Orders Consumed')
ax2.set_title('Orderbook Depth Consumed')
ax2.set_xticks(x)
ax2.set_xticklabels(impacts)
ax2.legend()

plt.tight_layout()
plt.show()

print("\n💰 Manipulation Cost Summary:")
display(cost_df.round(0))

## 3. Leveraged Position Profit Analysis

With the ability to manipulate spot prices, attackers can profit from leveraged perpetual positions. We analyze the economics of combined spot manipulation + perpetual positions.

### Key Parameters:
- **Leverage**: Amplifies both profits and liquidation risk
- **Position Size**: Larger positions capture more profit but require more margin
- **Holding Period**: Longer periods accumulate funding costs

In [ ]:
# Analyze manipulation profitability with different parameters
leverage_levels = [5, 10, 25, 50]
position_sizes = [100000, 500000, 1000000, 5000000]  # USD
target_impact = 5  # 5% price manipulation

# Get manipulation cost for target impact
manip_cost = calculate_manipulation_cost(orderbook, target_impact, 'up')['manipulation_cost_usd']

# Create profitability matrix
profit_matrix = pd.DataFrame(index=leverage_levels, columns=[f'${s/1000:.0f}k' for s in position_sizes])

for lev in leverage_levels:
    for size in position_sizes:
        pnl = analyze_manipulation_pnl(
            manipulation_cost=manip_cost,
            price_impact_pct=target_impact,
            leverage=lev,
            position_size_usd=size,
            funding_rate=0.0001  # 0.01% per period
        )
        profit_matrix.loc[lev, f'${size/1000:.0f}k'] = pnl['net_profit']

# Convert to numeric for heatmap
profit_matrix_numeric = profit_matrix.astype(float)

# Create heatmap
fig, ax = plt.subplots(figsize=(10, 6))
im = ax.imshow(profit_matrix_numeric.values, cmap='RdYlGn', aspect='auto')

# Set ticks
ax.set_xticks(np.arange(len(position_sizes)))
ax.set_yticks(np.arange(len(leverage_levels)))
ax.set_xticklabels(profit_matrix.columns)
ax.set_yticklabels([f'{l}x' for l in leverage_levels])

# Add colorbar
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('Net Profit ($)', rotation=270, labelpad=20)

# Add text annotations
for i in range(len(leverage_levels)):
    for j in range(len(position_sizes)):
        profit = profit_matrix_numeric.iloc[i, j]
        text = ax.text(j, i, f'${profit/1000:.0f}k',
                       ha="center", va="center", 
                       color="white" if abs(profit) > 50000 else "black")

ax.set_title(f'Manipulation Profitability Matrix (5% Price Impact)\nManipulation Cost: ${manip_cost/1000:.0f}k')
ax.set_xlabel('Position Size')
ax.set_ylabel('Leverage')

plt.tight_layout()
plt.show()

## 4. Funding Rate Manipulation

Attackers can create position imbalances to drive funding rates to extremes, extracting payments from other traders. This is often more subtle than price manipulation.

### Funding Manipulation Strategy:
1. Create large position imbalance
2. Drive funding rate to maximum
3. Collect funding payments
4. Unwind when profitable

In [ ]:
# Analyze funding rate manipulation scenarios
imbalance_sizes = [1e6, 5e6, 10e6, 25e6, 50e6]  # USD imbalances
funding_results = []

for imbalance in imbalance_sizes:
    result = analyze_funding_manipulation(
        orderbook=orderbook,
        position_imbalance_usd=imbalance,
        current_funding=0.0001,  # 0.01% baseline
        max_funding=0.01  # 1% max per period
    )
    funding_results.append(result)

funding_df = pd.DataFrame(funding_results)

# Visualize funding manipulation impact
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Funding rate impact
ax1.plot(funding_df['position_imbalance_usd']/1e6, 
         funding_df['new_funding_rate']*100, 
         color=COLORS['primary'], marker='o', linewidth=2.5)
ax1.axhline(y=0.01, color='gray', linestyle='--', label='Baseline Rate')
ax1.axhline(y=1.0, color='red', linestyle='--', label='Max Rate Cap')
ax1.set_xlabel('Position Imbalance ($MM)')
ax1.set_ylabel('Funding Rate (%)')
ax1.set_title('Funding Rate vs Position Imbalance')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Daily funding cost
ax2.bar(funding_df['position_imbalance_usd']/1e6, 
        funding_df['daily_funding_cost']/1000,
        color=COLORS['warning'])
ax2.set_xlabel('Position Imbalance ($MM)')
ax2.set_ylabel('Daily Funding Cost ($000s)')
ax2.set_title('Cost to Maintain Imbalance')
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\n📊 Funding Manipulation Analysis:")
display(funding_df[['position_imbalance_usd', 'imbalance_ratio', 'new_funding_rate', 'daily_funding_cost']].round(2))

## 5. Oracle Defense Mechanisms

Robust oracle design is the primary defense against manipulation. We analyze how multi-venue price feeds and time-weighted averaging protect against attacks.

### Oracle Defense Layers:
1. **Multi-Venue Aggregation**: Requires manipulating multiple markets
2. **TWAP/VWAP**: Increases cost of sustained manipulation
3. **Outlier Detection**: Filters extreme price deviations
4. **Circuit Breakers**: Halts trading during anomalies

In [ ]:
# Simulate oracle defense effectiveness
spot_venues = [
    {'name': 'Binance', 'liquidity_usd': 10e6},
    {'name': 'Coinbase', 'liquidity_usd': 8e6},
    {'name': 'Kraken', 'liquidity_usd': 5e6},
    {'name': 'OKX', 'liquidity_usd': 7e6},
    {'name': 'Uniswap', 'liquidity_usd': 3e6}
]

manipulation_targets = [1, 2, 5, 10]  # Percentage targets
oracle_analysis = []

for target in manipulation_targets:
    defense = analyze_oracle_defense(spot_venues, target)
    defense['target_impact'] = target
    oracle_analysis.append(defense)

oracle_df = pd.DataFrame(oracle_analysis)

# Visualize oracle defense
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Cost comparison
width = 0.35
x = np.arange(len(manipulation_targets))
ax1.bar(x - width/2, oracle_df['manipulation_cost_single']/1e6, 
        width, label='Single Venue', color=COLORS['danger'])
ax1.bar(x + width/2, oracle_df['manipulation_cost_all']/1e6, 
        width, label='All Venues', color=COLORS['success'])
ax1.set_xlabel('Target Impact (%)')
ax1.set_ylabel('Manipulation Cost ($MM)')
ax1.set_title('Cost to Manipulate: Single vs Multi-Venue Oracle')
ax1.set_xticks(x)
ax1.set_xticklabels(manipulation_targets)
ax1.legend()

# TWAP defense visualization
ax2.plot(oracle_df['target_impact'], 
         oracle_df['sustained_manipulation_cost']/1e6,
         color=COLORS['primary'], marker='o', linewidth=2.5)
ax2.fill_between(oracle_df['target_impact'], 
                 0, oracle_df['sustained_manipulation_cost']/1e6,
                 alpha=0.3, color=COLORS['primary'])
ax2.set_xlabel('Target Impact (%)')
ax2.set_ylabel('Sustained Cost ($MM)')
ax2.set_title('Cost to Sustain Manipulation (5min TWAP)')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n🛡️  Oracle Defense Analysis:")
display(oracle_df[['target_impact', 'manipulatable_venues', 'manipulation_difficulty', 'oracle_resistant']].round(2))

## 6. Combined Attack Scenarios

Sophisticated attackers may combine multiple vectors for maximum impact. We analyze complex scenarios combining price manipulation, leveraged positions, and funding rate gaming.

### Attack Combination Matrix

Different attack combinations have varying risk-reward profiles:

In [ ]:
# Define combined attack scenarios
scenarios = {
    'Simple Pump': {
        'spot_manipulation': 5,
        'leverage': 10,
        'position_size': 1e6,
        'funding_game': False
    },
    'Cascade Hunt': {
        'spot_manipulation': 10,
        'leverage': 25,
        'position_size': 5e6,
        'funding_game': False
    },
    'Funding Squeeze': {
        'spot_manipulation': 2,
        'leverage': 5,
        'position_size': 10e6,
        'funding_game': True
    },
    'Full Attack': {
        'spot_manipulation': 10,
        'leverage': 50,
        'position_size': 10e6,
        'funding_game': True
    }
}

scenario_results = []

for name, params in scenarios.items():
    # Calculate costs and profits
    manip_cost = calculate_manipulation_cost(
        orderbook, params['spot_manipulation'], 'up'
    )['manipulation_cost_usd']
    
    pnl = analyze_manipulation_pnl(
        manipulation_cost=manip_cost,
        price_impact_pct=params['spot_manipulation'],
        leverage=params['leverage'],
        position_size_usd=params['position_size']
    )
    
    # Add funding game profits if applicable
    funding_profit = 0
    if params['funding_game']:
        funding_analysis = analyze_funding_manipulation(
            orderbook=orderbook,
            position_imbalance_usd=params['position_size'],
            current_funding=0.0001,
            max_funding=0.01
        )
        # Assume collect funding for 24 hours
        funding_profit = params['position_size'] * funding_analysis['new_funding_rate'] * 3
    
    total_profit = pnl['net_profit'] + funding_profit
    
    scenario_results.append({
        'Scenario': name,
        'Manipulation Cost': manip_cost,
        'Position Profit': pnl['position_pnl'],
        'Funding Profit': funding_profit,
        'Total Profit': total_profit,
        'ROI %': (total_profit / pnl['total_capital_required']) * 100,
        'Risk Level': 'Extreme' if params['leverage'] > 25 else 'High' if params['leverage'] > 10 else 'Medium'
    })

scenario_df = pd.DataFrame(scenario_results)

# Visualize scenario comparison
fig, ax = plt.subplots(figsize=(12, 6))

scenarios_list = scenario_df['Scenario'].tolist()
x = np.arange(len(scenarios_list))
width = 0.25

ax.bar(x - width, scenario_df['Manipulation Cost']/1000, 
       width, label='Manipulation Cost', color=COLORS['danger'])
ax.bar(x, scenario_df['Position Profit']/1000, 
       width, label='Position Profit', color=COLORS['success'])
ax.bar(x + width, scenario_df['Total Profit']/1000, 
       width, label='Total Profit', color=COLORS['primary'])

ax.set_xlabel('Attack Scenario')
ax.set_ylabel('Amount ($000s)')
ax.set_title('Manipulation Scenario Analysis')
ax.set_xticks(x)
ax.set_xticklabels(scenarios_list, rotation=45)
ax.legend()
ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)

plt.tight_layout()
plt.show()

print("\n⚔️  Combined Attack Scenario Analysis:")
display(scenario_df.round(0))

## 7. Defense Recommendations

Based on our manipulation simulations, we recommend a multi-layered defense strategy:

### Technical Defenses

1. **Oracle Hardening**
   - Use 5+ independent spot venues
   - Implement 5-minute TWAP/VWAP
   - Add outlier detection (>3σ rejection)
   - Circuit breakers at 10% deviations

2. **Position Limits**
   - Dynamic limits based on market depth
   - Tier limits by trader history
   - Aggregate exposure monitoring
   - Cross-market position tracking

3. **Funding Rate Controls**
   - Cap maximum funding at 0.5% per period
   - Smooth funding transitions
   - Alert on rapid funding changes
   - Consider funding auctions

### Economic Defenses

1. **Liquidity Incentives**
   - Reward market makers during volatility
   - Penalty for quote stuffing
   - Depth requirements for makers
   - Cross-venue liquidity sharing

2. **Fee Structure**
   - Higher fees for aggressive orders
   - Volume-based fee discounts
   - Anti-manipulation fee surcharge
   - Time-in-market fee benefits

### Monitoring & Response

1. **Real-Time Detection**
   - Anomaly detection algorithms
   - Cross-market correlation monitoring
   - Funding rate deviation alerts
   - Large position notifications

2. **Incident Response**
   - Automated circuit breakers
   - Manual intervention protocols
   - Post-incident analysis
   - Continuous improvement process

In [ ]:
# Generate defense parameter recommendations
defense_params = {
    'Oracle Configuration': {
        'Min Venues': 5,
        'TWAP Window': '5 minutes',
        'Outlier Threshold': '3 sigma',
        'Circuit Breaker': '10%'
    },
    'Position Controls': {
        'Max Leverage': 50,
        'Position Limit': '25% of 1% depth',
        'OI Cap': '50% of daily volume',
        'Concentration Limit': '10% of total OI'
    },
    'Funding Controls': {
        'Max Rate': '0.5% per period',
        'Smoothing Period': '1 hour',
        'Alert Threshold': '0.1% change',
        'Emergency Cap': '1% per period'
    },
    'Economic Incentives': {
        'Maker Rebate': '0.02%',
        'Taker Fee': '0.05%',
        'Depth Bonus': '0.01% for >$1M',
        'Stability Fund': '10% of fees'
    }
}

print("\n🛡️  Recommended Defense Parameters:")
for category, params in defense_params.items():
    print(f"\n{category}:")
    for param, value in params.items():
        print(f"  - {param}: {value}")

# Calculate defense effectiveness
print("\n✅ Defense Effectiveness Estimates:")
print("- Multi-venue oracle: Increases manipulation cost by 300-500%")
print("- TWAP implementation: Increases sustained attack cost by 10-25x")
print("- Position limits: Reduces maximum extractable value by 75%")
print("- Combined defenses: Make profitable manipulation nearly impossible")